In [1]:
# Importing packages

import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss, brier_score_loss
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix

from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

from sklearn.metrics import r2_score


In [2]:
dataTrain = pd.concat([pd.read_csv("dataTrain_9.csv"),pd.read_csv("dataTrain_24.csv")], ignore_index=True)

dataTrain = dataTrain.drop(dataTrain[dataTrain.age > 75].index)
dataTrain = dataTrain.drop(dataTrain[dataTrain.carValue > 50000].index)
dataTrain = dataTrain.drop(dataTrain[dataTrain.exposure < 90 ].index)

print(dataTrain['claimNumber'].value_counts())

for col_name in ["gender", "carType","carCategory", "occupation", "carGroup","subRegion","region"]:
    if(dataTrain[col_name].dtype == 'object'):
        dataTrain[col_name]= dataTrain[col_name].astype('category')

dataTrain = dataTrain.drop(columns='id')
print(dataTrain)

from sklearn.preprocessing import LabelEncoder

columnsToEncode = list(dataTrain.select_dtypes(include=['category','object']))

for feature in columnsToEncode:
    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(dataTrain[feature])
# Drop column B as it is now encoded
    dataTrain = dataTrain.drop(feature,axis = 1)
# Join the encoded df
    dataTrain = dataTrain.join(one_hot)

0    91278
1    13905
2     2190
3      482
4       87
5       35
6       17
7        5
8        1
Name: claimNumber, dtype: int64
        gender carType carCategory     occupation  age  carGroup  bonus  \
0       Female       D       Small       Employed   23        19    120   
1         Male       E       Small     Unemployed   26         1     30   
3         Male       A       Small  Self-employed   30         1    -40   
4         Male       B       Small  Self-employed   35         1     60   
5       Female       D       Small       Employed   40         8     50   
...        ...     ...         ...            ...  ...       ...    ...   
119995    Male       D      Medium  Self-employed   39        12    -30   
119996    Male       E       Large        Retired   61         2     10   
119997    Male       C      Medium     Unemployed   37        20    130   
119998    Male       A      Medium        Retired   46        17    -10   
119999    Male       B      Medium       Emp

In [3]:
X = dataTrain.loc[:, (dataTrain.columns != "claimValue") & (dataTrain.columns != "claimNumber") & (dataTrain.columns != "Female")]

y_freq = dataTrain["claimNumber"]
y_severity = dataTrain["claimValue"]
    # First Stratification because the dataset is imbalanced

X_train, X_test, y_train_freq, y_test_freq = train_test_split(X, y_freq, 
#stratify=y_freq, 
test_size = 0.000001, random_state = 41)
X_train, X_test, y_train_severity, y_test_severity = train_test_split(X, y_severity, 
# stratify=y_freq, 
test_size = 0.000001, random_state = 41)

In [4]:
# Feature engineering on X_train and y_train to help model

    # Assembling X_train and y_train

X_train = pd.concat([X_train, y_train_freq, y_train_severity], axis=1)


In [5]:
# Multiplying according to y_train (=claimNumber)

## Faire une maille plus grande (feature engineering)

X_train = X_train.loc[X_train["claimNumber"]==0].append(X_train.loc[X_train.index.repeat(X_train.claimNumber)], ignore_index = True)

X_train['claimNumber'][X_train['claimNumber'] > 0] = 1

# Remelanger la base Train car index.repeat a classe les individus par leur claimNumber

X_train = X_train.sample(frac = 1)

    # Dissociate X_train and y_train
y_train_freq = X_train["claimNumber"]
y_train_severity = X_train["claimValue"]
weight_train = X_train["exposure"]
X_train = X_train.loc[:, (X_train.columns != "claimNumber") & (X_train.columns != "claimValue") & (X_train.columns != "exposure")]

# Changing y_test

y_test_freq[y_test_freq > 0] = 1

/var/folders/gj/vvd1q9td5wx3hwy02b2qvb0r0000gn/T/ipykernel_71962/2298949668.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['claimNumber'][X_train['claimNumber'] > 0] = 1


In [6]:
from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(max_features=5, n_estimators=100)
# rf = RandomForestClassifier(n_estimators=100, class_weight = "balanced", max_depth=15, max_features=0.6, min_samples_leaf = 7)

rf = RandomForestClassifier(n_estimators=300,class_weight = {0: 1, 1: 7}, max_depth=30, max_features=0.8, min_samples_leaf = 4, warm_start = True, n_jobs = 4)

rf.fit(X_train, y_train_freq, sample_weight=weight_train)


RandomForestClassifier(class_weight={0: 1, 1: 7}, max_depth=30,
                       max_features=0.8, min_samples_leaf=4, n_estimators=300,
                       n_jobs=4, warm_start=True)

In [7]:
X_train["prob"] = rf.predict_proba(X_train)[:,1]
X_test["prob"] = rf.predict_proba(X_test.loc[:, (X_test.columns != "exposure")])[:,1]

/var/folders/gj/vvd1q9td5wx3hwy02b2qvb0r0000gn/T/ipykernel_71962/2682224860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["prob"] = rf.predict_proba(X_test.loc[:, (X_test.columns != "exposure")])[:,1]


In [8]:
# Regression

rfreg = RandomForestRegressor(n_estimators = 110, warm_start = True, max_depth = 20, 
max_features=0.8, max_samples=0.9,
n_jobs = 4)

rfreg.fit(X_train, y_train_severity, sample_weight = weight_train)

RandomForestRegressor(max_depth=20, max_features=0.8, max_samples=0.9,
                      n_estimators=110, n_jobs=4, warm_start=True)

In [9]:
## Done with models

In [10]:
# Working on the dataTest set

dataTest = pd.read_csv("test.csv")
for col_name in ["gender", "carType","carCategory", "occupation", "carGroup","subRegion","region"]:
    if(dataTest[col_name].dtype == 'object'):
        dataTest[col_name]= dataTest[col_name].astype('category')

dataTest = dataTest.drop(columns='id')

columnsToEncode = list(dataTest.select_dtypes(include=['category','object']))
categories = []

for feature in columnsToEncode:
    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(dataTest[feature])
# Drop column B as it is now encoded
    dataTest = dataTest.drop(feature,axis = 1)
# Join the encoded df
    dataTest = dataTest.join(one_hot)
    categories = categories + list(one_hot.columns)

X_ = dataTest.loc[:, (dataTest.columns != "Female")]

# Make predictions with probability estimates
probs_test = rf.predict_proba(X_)

X_['prob'] = probs_test[:,1]

In [14]:
# Calculate the premium

premium = rfreg.predict(X_)

In [15]:
# Adjusting the premium 

# print(len(premium))

# print(sum(premium)/len(premium))

premium[premium < 200] = 210

# print(sum(premium)/len(premium))

premium[(premium > 400)&(premium < 600)] += 50

# print(sum(premium)/len(premium))

premium[premium < 800] += 30

# print(sum(premium)/len(premium))

premium += 70

print(sum(premium)/len(premium))
#premium = premium + 70

425.9347474183762


In [16]:
premium = pd.DataFrame(premium)
premium.rename(columns = {0:'premium'}, inplace = True)
premium.index += 1
premium.to_csv(r'/Users/tina/Desktop/Projet Data Science/premium_9.csv', index=True, header=True, index_label='index')